# This note book was made as a part of ML course in ITI internship ( for educational purpose ) 
steps done in this note book are
- data
 - about data
 - data summary
 
- feature engineering
 - missing value analysis
 - outliers analysis
 - filling 0s in wind spped column using RandomForest
 
- data visualization
 - visualize the distripution
 - visualize target vs features
 
- modeling
 - try linear model
 - regularization model
 - ensemble mdoel

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sb
sb.set()

In [ ]:
# /kaggle/input/bike-sharing-demand/sampleSubmission.csv
# /kaggle/input/bike-sharing-demand/train.csv
# /kaggle/input/bike-sharing-demand/test.csv

# <center>import data

In [ ]:
train_data = pd.read_csv('/kaggle/input/bike-sharing-demand/train.csv')
test_data = pd.read_csv('/kaggle/input/bike-sharing-demand/test.csv')

In [ ]:
train_data.head()

In [ ]:
test_data.head()

In [ ]:
print(train_data.shape)
print(test_data.shape)

| data | #rows | #columns | |
| ---- | ----- | ------- | |
| train data | 10886 |  12 | |
|test data | 6493 |  9| there is no <br> casual<br>	registered<br>	count<br> columns in test |

# <center>plot data count

In [ ]:
fig , axs = plt.subplots(1,4 , figsize=(20,5) , sharey=False , sharex=False , )
fig.tight_layout(pad=2.5)
sb.countplot(data=train_data , x='season'      , ax=axs[0])
sb.countplot(data=train_data , x='holiday'     , ax=axs[1])
sb.countplot(data=train_data , x='workingday'  , ax=axs[2])
sb.countplot(data=train_data , x='weather'     , ax=axs[3])


so, both the training data and testing data has the following columns

| cloumn name | discription | values |
| ----------- | ----------- | ----------- |
| datetime | time stamp | YYYY-MM-DD HH-mm-ss format | 
| season   | - |  1 = spring, 2 = summer, 3 = fall, 4 = winter |
| holiday  |  whether the day is considered a holiday  | 1= holyday |
| workingday |  whether the day is neither a weekend nor holiday | 1= it's working day |
| weather | - | 1: Clear, Few clouds, Partly cloudy, Partly cloudy <br/> 2: Mist + Cloudy, Mist + Broken clouds, Mist + Few clouds, Mist <br/> 3: Light Snow, Light Rain + Thunderstorm + Scattered clouds, Light Rain + Scattered clouds <br/> 4: Heavy Rain + Ice Pallets + Thunderstorm + Mist, Snow + Fog 
 |
| casual | number of non-registered user rentals initiated | - |
| registered | number of registered user rentals initiated | - |
| **count** | number of total rentals | target |


from this data we can say that
- 

| cloumn name | needed or not | |
| ----------- | ----------- | |
| datetime | neede to be splited  | |
| season   | needed | |
| holiday  |  not needed  | the working day also represents <br> this column |
| workingday |  needed | |
| weather	| | |
| temp	| | |
| atemp	| | |
| humidity	| | |
| windspeed | | |
| casual | not needed | the count cannot be a feature <br> as it should be predicted |
| registered | not needed | the count cannot be a feature <br> as it should be predicted |
| **count** | target | |

# <center>split datetime column transformer

In [ ]:
from sklearn.base import TransformerMixin , BaseEstimator
from datetime import datetime
class TSSplit(BaseEstimator , TransformerMixin):
    def __init__(self , format_str , out_col = ['year' , 'month' , 'day' , 'hour' , 'minute' , 'sec' , 'dayname' , 'dayNUM' , 'isweekend'] , weed_end_days = ['Saturday' ,'Sunday']):
        self.format_str    = format_str
        self.out_col       = out_col
        self.weed_end_days = weed_end_days
    
    def fit(self , X , y=None):
        return self
    
    def columns(self):
        return self.out_col
    
    def transform(self , X):
        col = X.copy().values
        dict = {}
        
        for i in self.out_col:
            if i == 'day'      : dict['day'       ] = np.array([datetime.strptime(i[0], self.format_str).day                                  for i in col] ).reshape(-1,1)
            if i == 'year'     : dict['year'      ] = np.array([datetime.strptime(i[0], self.format_str).year                                 for i in col] ).reshape(-1,1)
            if i == 'month'    : dict['month'     ] = np.array([datetime.strptime(i[0], self.format_str).month                                for i in col] ).reshape(-1,1)
            if i == 'hour'     : dict['hour'      ] = np.array([datetime.strptime(i[0], self.format_str).hour                                 for i in col] ).reshape(-1,1)
            if i == 'minute'   : dict['minute'    ] = np.array([datetime.strptime(i[0], self.format_str).minute                               for i in col] ).reshape(-1,1)
            if i == 'sec'      : dict['sec'       ] = np.array([datetime.strptime(i[0], self.format_str).second                               for i in col] ).reshape(-1,1)
            if i == 'dayname'  : dict['dayname'   ] = np.array([datetime.strptime(i[0], self.format_str).strftime("%A")                       for i in col] ).reshape(-1,1)
            if i == 'dayNUM'   : dict['dayNUM'    ] = np.array([datetime.strptime(i[0], self.format_str).isoweekday()                            for i in col] ).reshape(-1,1)
            if i == 'isweekend': dict['isweekend' ] = np.array([datetime.strptime(i[0], self.format_str).strftime("%A") in self.weed_end_days for i in col] ).astype(int).reshape(-1,1)
        
        out_list = []
        
        for i in self.out_col:
            out_list.append(dict[i])
        return np.concatenate(out_list , axis=1)
  

# exmple
# from sklearn.compose import ColumnTransformer

# obj = TSSplit(format_str='%Y-%m-%d %H:%M:%S')
# out = obj.fit_transform(train_data)

# pd.DataFrame(out , columns=obj.columns()).head(50)

# <center>cluster weather transformer

In [ ]:
from sklearn.base import TransformerMixin , BaseEstimator
from sklearn.cluster import DBSCAN , KMeans 

from datetime import datetime

class LablizeGroup(BaseEstimator , TransformerMixin):
    def __init__(self  , df , columns , K = 5 ):
        self.K    = K
        self.df  = df
        self.columns = columns
    
    def fit(self , X=None , y=None):
        self.clustrer = KMeans(n_clusters = self.K , random_state=42)
        return self
        
    def getLablesFromTest(self , test_df):
        data_to_get_label = test_df[self.columns]
        labels = self.clustrer.predict(data_to_get_label)
        return pd.DataFrame(labels , columns=['lables'])
    
    def transform(self , X):
        data_to_cluster = self.df[self.columns]
        labels          = self.clustrer.fit_predict(data_to_cluster)
        return pd.DataFrame(self.clustrer.labels_ , columns=['lables'])


# <center>generate year , month , day , hour , minute , sec , dayname , dayNUM , isweekend

In [ ]:
TSSplit_obj = TSSplit(format_str='%Y-%m-%d %H:%M:%S' , weed_end_days=['Saturday', 'Sunday'] )
res1 = TSSplit_obj.fit_transform(train_data)
cols_outofDate = pd.DataFrame(res1 , columns=TSSplit_obj.columns())
cols_outofDate

# <center>concatinate year , month , ... cols with other columns

In [ ]:
trainWithDate = pd.concat(
    [
        train_data.drop(['datetime'] , axis=1  ) ,
        cols_outofDate , 
    ], axis=1
)
pd.set_option('display.max_columns' , None)
trainWithDate

# <center>replace zeros in windspeed column using randomforest

In [ ]:
trainWithDate[trainWithDate['windspeed'] == 0]

In [ ]:
from sklearn.base import TransformerMixin , BaseEstimator
from sklearn.ensemble import RandomForestRegressor , BaggingRegressor , ExtraTreesRegressor

from datetime import datetime

class Replace0_sUsingRF():
    def __init__(self  , df  , columnwithzeros , X_cols = []):
        self.df  = df
        self.columnwithzeros = columnwithzeros
        self.X_cols = X_cols
    
    def fit(self):
        self.RF = RandomForestRegressor(max_depth = 1000 , random_state=42)
        self.RF = ExtraTreesRegressor(random_state=42)
        return self
        
    
    def transform(self):
        datawith0    = self.df[self.df[self.columnwithzeros]==0]
        datawithout0 = self.df[self.df[self.columnwithzeros]!=0]
        self.RF.fit(datawithout0[self.X_cols] , datawithout0[self.columnwithzeros])
        predictions = pd.DataFrame(self.RF.predict(datawith0[self.X_cols]).reshape(-1,1) , index=datawith0.index)
        imputed_data  =  pd.merge(datawith0.drop([self.columnwithzeros] , axis=1) , pd.DataFrame(predictions), on=datawith0.index)
        imputed_data.rename(columns={ 0 : self.columnwithzeros} , inplace=True)
        return pd.concat([datawithout0 , imputed_data.set_index('key_0')] , axis=0).sort_index()
    
dfwithout0 = Replace0_sUsingRF(trainWithDate ,X_cols=["season","weather","humidity","month","temp","year","atemp"] , columnwithzeros='windspeed').fit().transform()

In [ ]:
fig , axs = plt.subplots(1 , 2 , figsize=(10 , 5))
sb.histplot(data=trainWithDate , x='windspeed' , ax=axs[0] , kde=True)
sb.histplot(data=dfwithout0    , x='windspeed' , ax=axs[1] , kde=True)

In [ ]:
fig , axs = plt.subplots(1 , 2 , figsize=(10 , 5))
sb.histplot(data=dfwithout0    , x='windspeed' , ax=axs[1] , kde=True)
ndf = dfwithout0.copy()
ndf['windspeed'] = np.sqrt(ndf['windspeed'])
sb.histplot(data=ndf , x='windspeed' , ax=axs[0] , kde=True)


In [ ]:
# dfwithout0['windspeed'] = np.sqrt(dfwithout0['windspeed'])

# <center>clustering weather data

In [ ]:
LablizeGroup_obj = LablizeGroup(dfwithout0 , ['temp','atemp','humidity','windspeed'] , K = 4)
res2 = LablizeGroup_obj.fit_transform(dfwithout0)
labels = pd.DataFrame(res2)
labels

In [ ]:
sb.countplot(data = labels , x='lables')

# <center>merge all together

In [ ]:
fedf = pd.concat(
    [
        dfwithout0 ,
        labels
    ], axis=1
)
pd.set_option('display.max_columns' , None)
fedf

# <center> Missing Values Analysis

In [ ]:
import missingno as msno
msno.matrix(fedf,figsize=(12,5))

In [ ]:
# season	holiday	workingday	weather	temp	atemp	humidity	windspeed	casual	registered	count	year	month	day	hour	minute	sec	dayname	dayNUM	isweekend	lables
fig , axs = plt.subplots(3,4 , figsize=(30 , 20))
fig.tight_layout(pad=2.5)
sb.boxplot(data=fedf , y='count' , x='season'  , orient='v' , ax=axs[0][0] )
sb.boxplot(data=fedf , y='count' , x='holiday'  , orient='v' , ax=axs[0][1] )
sb.boxplot(data=fedf , y='count' , x='workingday'  , orient='v' , ax=axs[0][2] )
sb.boxplot(data=fedf , y='count' , x='weather'  , orient='v' , ax=axs[0][3] )

sb.boxplot(data=fedf , y='count' , x='year'  , orient='v' , ax=axs[1][0] )
sb.boxplot(data=fedf , y='count' , x='month'  , orient='v' , ax=axs[1][1] )
sb.boxplot(data=fedf , y='count' , x='day'  , orient='v' , ax=axs[1][2] )
sb.boxplot(data=fedf , y='count' , x='hour'  , orient='v' , ax=axs[1][3] )

sb.boxplot(data=fedf , y='count' ,  orient='v' , ax=axs[2][0] )
sb.boxplot(data=fedf , y='count' , x='dayNUM'  , orient='v' , ax=axs[2][1] )
sb.boxplot(data=fedf , y='count' , x='isweekend'  , orient='v' , ax=axs[2][2] )
sb.boxplot(data=fedf , y='count' , x='lables'  , orient='v' , ax=axs[2][3] )

as we can see there is outliers in count column so it needs further investigation

In [ ]:
df_olDroped = fedf[fedf['count'] < (( fedf['count'].quantile(q=0.75) - fedf['count'].quantile(q=0.25) ) * 1.5)]
# df_olDroped = fedf[np.abs(fedf["count"]-fedf["count"].mean())<=(3*fedf["count"].std())] 

In [ ]:
print(fedf.shape)
print(df_olDroped.shape)

In [ ]:
df_olDroped

In [ ]:
sb.boxplot(data=df_olDroped , y='count' ,  orient='v'  )

In [ ]:
fig, ax = plt.subplots(figsize=(16,15))
sb.heatmap(data=df_olDroped.corr(), annot=True, linewidths=.5, ax=ax)

In [ ]:
from sklearn.model_selection import train_test_split
X_train , X_test , y_train , y_test = train_test_split(df_olDroped.drop(columns=['count' , 'dayname' , 	'registered' ,'casual' , 'year' , 'day'  , 'temp' , 'minute',	'sec' ] ,axis=1) , df_olDroped['count'] , test_size = 0.1)

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor , AdaBoostRegressor , BaggingRegressor
model = BaggingRegressor()
model.fit(X_train , y_train)
print(model.score(X_train , y_train))
print(model.score(X_test , y_test))

In [ ]:
from sklearn.metrics import mean_squared_log_error
train_err = np.sqrt(mean_squared_log_error(y_train , model.predict(X_train)))
valid_err = np.sqrt(mean_squared_log_error(y_test  , model.predict(X_test )))
print('training error = ' , train_err)
print('testing error = ' , valid_err)
print(train_err - valid_err*100)

In [ ]:
test_data


In [ ]:
test_data = pd.read_csv('/kaggle/input/bike-sharing-demand/test.csv')
test_data

TSSplit_obj = TSSplit(format_str='%Y-%m-%d %H:%M:%S' , weed_end_days=['Saturday', 'Sunday'] )
res1 = TSSplit_obj.fit_transform(test_data)
cols_outofDate = pd.DataFrame(res1 , columns=TSSplit_obj.columns())
cols_outofDate


LablizeGroup_obj 
res2 = LablizeGroup_obj.getLablesFromTest(test_data)
labels = pd.DataFrame(res2 )
labels


fedf = pd.concat(
    [
        test_data.drop(['datetime'] , axis = 1),
        cols_outofDate ,
        labels
    ], axis=1
)
pd.set_option('display.max_columns' , None)
fedf

tst = fedf.drop(columns=['dayname'  , 'year' , 'day'  , 'temp' , 'minute',	'sec' ] ,axis=1)

In [ ]:
pd.DataFrame(model.predict(tst) , columns=['count'] , index = test_data.datetime).to_csv('Submission.csv')